# Embedings precomputing

### Install dependancies and mount GDrive


In [4]:
# Check if running in Google Colab to run dumb-colab speciefic code
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    COLAB = True
else:
    print('Not running on CoLab')
    COLAB=False


Not running on CoLab


In [7]:
# Other than torch dependencies
# ! pip install fair-esm # Switched to Ankh embeddings
! pip install GPUtil
! pip install pynvml
! pip install ankh
! pip install tqdm

In [5]:
import torch
print("Pytorch " + torch.__version__)
# import esm
import pandas as pd
import numpy as np
import time
from GPUtil import showUtilization as gpu_usage

import ankh

from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import Trainer, TrainingArguments, EvalPrediction
from datasets import load_dataset

from sklearn import metrics
from scipy import stats
from functools import partial
from tqdm.auto import tqdm


# Create device agnostic code
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

Pytorch 1.13.1


'cuda'

### Load data from TSV


In [6]:
if COLAB:
    # Mount GDrive for Colab
    from google.colab import drive
    drive.mount('/content/drive')
    # Navigate Colab
    %cd /content/drive/MyDrive/Colab Notebooks/esm
    %ls

In [7]:
# Download dataset ~56MB
import requests

url = 'https://raw.githubusercontent.com/Dart-ilder/Cas_deep_search/main/esm/emb_pregen/cas_dataset_kira.tsv'
response = requests.get(url)

with open('./cas_dataset_kira.tsv', 'w') as file:
    file.write(response.text)

with open('./cas_dataset_kira.tsv') as file:
    cas_voc = pd.read_csv(file, delimiter="\t", comment='=')
cas_voc.head(3)

,Loci_id,Strand,Genome_assembly,Chromosome,N1,Protein_id,B2,Gene_id,Gene_family,Type,Species,Start,End,Seq,Prot
0,RN99_05230,-,GCA_001296125.1,CP012714.1,991.0,ALF19893_1,+,cd09644,csn2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100111,1100773,TTAGAAAATTTCACATAAATCATTATCTATAACAATTAAATTATCA...,MTFQYKGFNFKIDFEEKNIFSLIVENKRAYRKIIEDLVNNSNIEDG...
1,RN99_05235,-,GCA_001296125.1,CP012714.1,992.0,ALF20727_1,+,mkCas0206,cas2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100770,1101075,TCATAAAACCACAAGCCTTTCATCTGTTTCTAAAAATGTCCCTTTT...,MRMLLFFDLPSVTNSDLKEYRKFRKFLIENGFSMLQESVYSKLLLH...
2,RN99_05240,-,GCA_001296125.1,CP012714.1,993.0,ALF19894_1,+,cd09720,cas1,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1101080,1101958,CTATAACTCATCTTGAAAAAATCTCACTAATGATAAATCATTTGAG...,MSGWRVIIVTGRGKLDLRYNSISIRRDNGTDFIHIGEVNTLILETT...


# Model download

In [11]:
model, tokenizer = ankh.load_base_model()
model.eval()
model.to(DEVICE)

Some weights of the model checkpoint at ElnaggarLab/ankh-base were not used when initializing T5EncoderModel: ['decoder.block.23.layer.1.EncDecAttention.k.weight', 'decoder.block.7.layer.1.EncDecAttention.k.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.23.layer.2.layer_norm.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.7.layer.1.EncDecAttention.o.weight', 'decoder.block.12.layer.1.EncDecAttention.o.weight', 'decoder.block.10.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.13.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.0.SelfAttention.o.weight', 'decoder.block.11.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.7.layer.2.DenseReluDense.wo.weight', 'decoder.block.16.layer.0.layer_norm.weight', 'decoder.block.1.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.13.layer.0.SelfAttention.q.weight', 'decoder.block.19.layer.0.SelfAttention.k.weight', 'decoder.block.8.

T5EncoderModel(
  (shared): Embedding(144, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(144, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(64, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=3072, bias=False)
              (wi_1): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_featu

In [12]:
cas_voc.head(3)

,Loci_id,Strand,Genome_assembly,Chromosome,N1,Protein_id,B2,Gene_id,Gene_family,Type,Species,Start,End,Seq,Prot
0,RN99_05230,-,GCA_001296125.1,CP012714.1,991.0,ALF19893_1,+,cd09644,csn2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100111,1100773,TTAGAAAATTTCACATAAATCATTATCTATAACAATTAAATTATCA...,MTFQYKGFNFKIDFEEKNIFSLIVENKRAYRKIIEDLVNNSNIEDG...
1,RN99_05235,-,GCA_001296125.1,CP012714.1,992.0,ALF20727_1,+,mkCas0206,cas2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100770,1101075,TCATAAAACCACAAGCCTTTCATCTGTTTCTAAAAATGTCCCTTTT...,MRMLLFFDLPSVTNSDLKEYRKFRKFLIENGFSMLQESVYSKLLLH...
2,RN99_05240,-,GCA_001296125.1,CP012714.1,993.0,ALF19894_1,+,cd09720,cas1,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1101080,1101958,CTATAACTCATCTTGAAAAAATCTCACTAATGATAAATCATTTGAG...,MSGWRVIIVTGRGKLDLRYNSISIRRDNGTDFIHIGEVNTLILETT...


In [13]:
def cache_clear(): # In colab to clear GPU cache you need to wait some time after deleting tensor
    if COLAB:
        time.sleep(0.02)
    torch.cuda.empty_cache()
    
def gpu_util(): # To monitor how much more can we load GPU with data
    if DEVICE == "cuda":
        return torch.cuda.memory_reserved(DEVICE)/torch.cuda.get_device_properties(DEVICE).total_memory
    if DEVICE == "cpu":
        return 0

In [14]:
protein_sequences = list(cas_voc["Prot"].values)
tokens = tokenizer.batch_encode_plus(protein_sequences, 
                                        add_special_tokens=True, 
                                        padding=True,
                                        is_split_into_words=False, 
                                        return_tensors="pt")
print(tokens['input_ids'].shape)



torch.Size([36970, 1811])


In [15]:
cache_clear()
print(torch.cuda.memory_reserved(DEVICE))
print(torch.cuda.get_device_properties(DEVICE))
import gc
gc.collect()
gpu_util()

2015363072
_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11263MB, multi_processor_count=28)


0.17063588946842748

In [16]:
BATCH_SIZE = 15 # For GTX 1080ti
batched_tokens = torch.split(tokens['input_ids'], BATCH_SIZE)
batched_attention_masks = torch.split(tokens['attention_mask'], BATCH_SIZE)
len(batched_tokens)
batched_tokens[0].shape

torch.Size([15, 1811])

In [17]:
gpu_usage()
with torch.no_grad():
    for idx, batch in enumerate(batched_tokens):
        
        inputs = batch.to(DEVICE)
        embeddings = model(input_ids=inputs)[0]
        output = embeddings[:, 0].cpu()
        output = output.numpy()
        np.savetxt(f"./batched_embs/batch{idx}.txt", output)
        print(f"{idx}/{len(batched_tokens)}\t GPU Usage:{gpu_util()}")
        exit()
        del inputs
        del embeddings
        del output
        cache_clear()

gpu_usage()
gpu_util()
cache_clear()
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 | 26% | 23% |
0/2465	 GPU Usage:0.8382643435800688
1/2465	 GPU Usage:0.8382643435800688
2/2465	 GPU Usage:0.8382643435800688
3/2465	 GPU Usage:0.8382643435800688
4/2465	 GPU Usage:0.8382643435800688
5/2465	 GPU Usage:0.8382643435800688
6/2465	 GPU Usage:0.8382643435800688
7/2465	 GPU Usage:0.8382643435800688
8/2465	 GPU Usage:0.8382643435800688
9/2465	 GPU Usage:0.8382643435800688
10/2465	 GPU Usage:0.8382643435800688
11/2465	 GPU Usage:0.8382643435800688
12/2465	 GPU Usage:0.8382643435800688
13/2465	 GPU Usage:0.8382643435800688
14/2465	 GPU Usage:0.8382643435800688
15/2465	 GPU Usage:0.8382643435800688
16/2465	 GPU Usage:0.8382643435800688
17/2465	 GPU Usage:0.8382643435800688
18/2465	 GPU Usage:0.8382643435800688
19/2465	 GPU Usage:0.8382643435800688
20/2465	 GPU Usage:0.8382643435800688
21/2465	 GPU Usage:0.8382643435800688
22/2465	 GPU Usage:0.8382643435800688
23/2465	 GPU Usage:0.8382643435800688
24/2465	 GPU Usage:0.8382643435800688
25/

(15, 768)


### If Out Of Memory error:

In [ ]:
# In case of OOM unkomment all dels
# Utilization from model alone should be ~25%

del inputs
del batched_attention_masks
del batch 
del batched_tokens
del embeddings
del output
cache_clear()
print(torch.cuda.memory_reserved(DEVICE))
print(torch.cuda.get_device_properties(DEVICE))
import gc
gc.collect()
gpu_util()

# ESM leftovers. Legacy


In [47]:
# Load ESM-2 model
torch.cuda.empty_cache()
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# device = "cpu" # My GPU doesn't have enough VRAM
# model.to(device)
device

NameError: name 'esm' is not defined

### Preprocessing data and tokenisation

In [38]:
# Prepare data into format [ (label, seq), ]. We also cut * end of protein sequence symbol
# I leave Loci_id as an unicue identificator of an entry and Gene_family as a target label
data = list()
for id, seq in cas_voc.iterrows():
    data.append((f">{seq.Gene_id}|{seq.Gene_family}|{seq.Loci_id}", seq.Prot))


In [39]:
# Prepare data
# Takes in data in format of [ (label, seq), ] list. Applies tokens preprocessing
# Returns: only_lables_batched, only_seqs_batched, seq_tokenized_batched 
data = data[:]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1) # returns lengths of tokenized seqs without padding

#batch_tokens = batch_tokens.to(device) # batch_converter automatically detects and moves data to gpu. But mine has too little VRAM
#print(np.array(batch_tokens.to("cpu")).shape)


In [42]:
batch_tokens.shape

<function Tensor.type>

### Run the model and get sequence representations

In [ ]:
# Extract per-residue representations (on CPU)
# Makes sence to extract only lasta layer representations. For 650M model it's layer 33
model = model.to(device)
batch_tokens_slice = batch_tokens.to(device)
with torch.inference_mode():
    results = model(batch_tokens, repr_layers=[33])
token_representations = results["representations"][33].to("cpu") # 1280 dimentional (for 650M model) representations for each residue in each data entry
token_representations.cpu()
print(np.array(token_representations).shape)


OutOfMemoryError: ignored

In [ ]:
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
print(np.array(token_representations).shape)
print(batch_lens)
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
print(np.array(sequence_representations).shape)

(20, 1376, 1280)
tensor([ 222,  103,  294, 1376,  190,  350,  325,  437, 1084,  328,  323, 1126,
         418,  296,  339,  205,  230,  319,  243, 1125])
(20,)


<ipython-input-21-ae456d3c61a3>:8: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  print(np.array(sequence_representations).shape)
<ipython-input-21-ae456d3c61a3>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(sequence_representations).shape)
